In [ ]:
from bs4 import BeautifulSoup as bsoup
import requests as reqs
from datetime import datetime

# Season to capture games for:
season_to_parse = "https://fbref.com/en/squads/986a26c1/2019-2020/Northampton-Town-Stats"

In [ ]:
def parse_seasons(season_to_parse):
    match_links = []
    page = reqs.get(season_to_parse)
    season_page = bsoup(page.content, 'html.parser')
    find_links = season_page.find_all('td',attrs={"data-stat":"match_report"})
    for x in find_links:
        add = x.find_next('a').attrs['href']
        match_links.append("https://fbref.com" + str(add))
    return game_data(match_links)

In [ ]:
def game_data(match_links):
    match_dataset = {}
    for match in match_links[:1]:
        parse_page = bsoup(reqs.get(match).content, 'html.parser')
        # Date
        find_date = parse_page.find("div",{"class":"scorebox_meta"}).find('a').get_text()
        date_adj = datetime.strptime(find_date[find_date.find(' ') + 1:], '%B %d, %Y')
        match_dataset['Date'] = date_adj.strftime('%Y-%m-%d')
        # Team names - capture from 'title'
        find_teams = parse_page.find("title").get_text()
        team_one = find_teams[:(find_teams.find('vs.') - 1)]
        team_two = find_teams[(find_teams.find('vs.') + 4):find_teams.find('Match') - 1]
        match_dataset['Home'] = team_one
        match_dataset['Away'] = team_two
    return match_dataset

In [ ]:
# Run the whole gambit
parse_seasons(season_to_parse)